In [ ]:
# Solve the imshow dead kernel problem
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
'''
Start loading the data
'''
print('================== START LOADING DATA ==================')

In [ ]:
from PIL import Image
import os
import os.path

import torch.utils.data
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

In [ ]:
# Split the training set into a 80% training set and 20% validation set
import random

def split_huge_file(file,out1,out2,percentage=0.75,seed=2022):
    """Splits a file in 2 given the approximate `percentage` to go in the large file."""
    random.seed(seed)
    with open(file, 'r',encoding="utf-8") as fin, \
         open(out1, 'w') as foutBig, \
         open(out2, 'w') as foutSmall:

        for line in fin:
            r = random.random() 
            if r < percentage:
                foutBig.write(line)
            else:
                foutSmall.write(line)

In [ ]:
path = 'C:/Users/xiaow/OneDrive/Desktop/Spring 2022/Introduction to Machine Learning/Introduction-to-Machine-Learning/Task 3'
split_huge_file(os.path.join(path, f'train_triplets.txt'), 'train_triplets_splits.txt', 'val_triplets_splits.txt', percentage=0.8, seed=2022)

In [ ]:
# Image loader helper function
def default_image_loader(path):
    return Image.open(path).convert('RGB')

In [ ]:
# Dataset
im = Image.open(r"C:\Users\xiaow\OneDrive\Desktop\Spring 2022\Introduction to Machine Learning\Introduction-to-Machine-Learning\Task 3\food\00001.jpg")

In [ ]:
display(im)

In [ ]:
data = np.asarray(im)
data.shape

In [ ]:
im.resize((354,242))

In [ ]:
class TripletImageLoader(torch.utils.data.Dataset):
    def __init__(self, base_path, triplets_file_name, transform=None, loader=default_image_loader):
        """ base_path: The path contains the text file of the training triplets
            triplets_file_name: The text file with each line containing three integers, 
            where integer i refers to the i-th image in the filenames file.  
            Each line contains three integers (a triplet).
            For example, the triplet "00723 00478 02630" denotes that the dish in image "00723.jpg" is more similar in taste 
            to the dish in image "00478.jpg" than to the dish in image "02630.jpg" according to a human annotator.
         """
        self.base_path = base_path  
        triplets = []
        for line in open(triplets_file_name):
            triplets.append((line.split()[0], line.split()[1], line.split()[2])) # anchor, positive, negative
        self.triplets = triplets
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        path1, path2, path3 = self.triplets[index]
        img1 = self.loader(os.path.join(self.base_path, f'{path1}.jpg'))
        img2 = self.loader(os.path.join(self.base_path, f'{path2}.jpg'))
        img3 = self.loader(os.path.join(self.base_path, f'{path3}.jpg'))
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)

        return img1, img2, img3

    def __len__(self):
        return len(self.triplets)

In [ ]:
# Initialization: importing the packages that we will use
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Google colab offers time limited use of GPU for free

################# Configuration  ######################
IMAGE_SIZE = (242, 354) # bigger image size improves performance but makes training slower.

# Training parameters 
BATCH_SIZE = 120

In [ ]:
torch.cuda.is_available()

In [ ]:
# Dataset and Trasformations
import torchvision
import torchvision.transforms as transforms

############# Datasets and Dataloaders ################
transform_train = transforms.Compose([
    transforms.ToTensor(), # The output of torchvision datasets are PILImage images of range [0, 1].
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5), # we want our network to be robust over geometrical transformations that leave the image semantically invariant
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #  We transform them to Tensors of normalized range [-1, 1].
    # (mean, mean, mean) , (std, std, std): output[channel] = (input[channel] - mean[channel]) / std[channel]
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMAGE_SIZE),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMAGE_SIZE),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

path = 'C:/Users/xiaow/OneDrive/Desktop/Spring 2022/Introduction to Machine Learning/Introduction-to-Machine-Learning/Task 3/food'
train_dataset = TripletImageLoader(path.rstrip('\n'), 'train_triplets_splits.txt', transform=transform_train)
val_dataset = TripletImageLoader(path.rstrip('\n'), 'val_triplets_splits.txt', transform=transform_val)
test_dataset = TripletImageLoader(path.rstrip('\n'), 'test_triplets.txt', transform=transform_test)

In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

In [ ]:
test_dataset[0]

In [ ]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
len(train_loader)

In [ ]:
# Visualization of Dataset
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    plt.figure()
    plt.imshow(img.permute(1, 2, 0))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images_anchor, images_positive, images_negative = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images_anchor))
imshow(torchvision.utils.make_grid(images_positive))
imshow(torchvision.utils.make_grid(images_negative))

In [ ]:
'''
Data loaded
'''
print('================== DATA LOADED ==================')

In [ ]:
'''
Start constructing the network
'''
print('================== START CONSTRUCTING NETWORK ==================')

In [ ]:
import torch.optim
import torch.utils.data
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torch.utils.data

#########################NET##############################

#The backbone for the CNNS with shared weights
def backbone(**kwargs):
    """
    Construct a ResNet-101 model.
    Returns:
        Embeddingnet(model): The CNN with the specified model as its backbone is instantiated
    """
    #model = torch.hub.load('pytorch/vision:v1.7.1', 'resnet101', pretrained=True)
    model = models.resnet18(pretrained=True)
    #model = models.resnet34(pretrained=True)
    #model = models.vgg11_bn()
    #model = torch.hub.load('pytorch/vision:v0.8.2', 'alexnet', pretrained=True)
    #model = models.alexnet(pretrained=True)            #used in the paper
    #print('Layers',model.children)
    #model = models.resnet50(pretrained=True)
    #model = models.inception_v3(pretrained=True)
    #model = torchvision.models.resnet.ResNet(
        #torchvision.models.resnet.BasicBlock, [2, 1, 1, 1])

    return EmbeddingNet(model)

#The overall network consisting of three embedding nets with shared weights
class TripletNet(nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet
        #print(self.embeddingnet.children())

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_a = self.embeddingnet(a)

        # positive examples
        embedded_p = self.embeddingnet(p)

        # negative examples
        embedded_n = self.embeddingnet(n)

        return embedded_a, embedded_p, embedded_n

#The CNN used by Triplet Net with 'model' as its backbone and a final fully connected Layer
class EmbeddingNet(nn.Module):
    """EmbeddingNet using the specified model in backbone()."""

    def __init__(self, resnet):
        """Initialize EmbeddingNet model."""
        super(EmbeddingNet, self).__init__()
        # Everything excluding the last linear layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        num_ftrs =  resnet.fc.in_features
        self.fc1 = nn.Linear(num_ftrs, 1024)

    def forward(self, x):
        """Forward pass of EmbeddingNet."""
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [ ]:
# Construct a triplet net
import torch.optim
import torch.utils.data
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torch.utils.data

#########################NET##############################

#The backbone for the CNNS with shared weights
def FeatureExtractNET(**kwargs):
    """
    Construct a ResNet-101 model.
    Returns: The CNN for feature extraction with a fully connected layer
    """
    model = models.resnet18(pretrained=True)

    return EmbeddingNet(model)

#The CNN used by Triplet Net with 'model' as its backbone and a final fully connected Layer
class EmbeddingNet(nn.Module):
    """EmbeddingNet using the specified model in backbone()."""

    def __init__(self, resnet):
        """Initialize EmbeddingNet model."""
        super(EmbeddingNet, self).__init__()
        # Everything excluding the last linear layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        num_ftrs =  resnet.fc.in_features
        self.fc1 = nn.Linear(num_ftrs, 1024)

    def forward(self, x):
        """Forward pass of EmbeddingNet."""
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    
#The overall network consisting of three embedding nets with shared weights
class TripletNet(nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_a = self.embeddingnet(a)

        # positive examples
        embedded_p = self.embeddingnet(p)

        # negative examples
        embedded_n = self.embeddingnet(n)

        return embedded_a, embedded_p, embedded_n

In [ ]:
net = TripletNet(backbone())
net

In [ ]:
batch = next(iter(train_loader))
net(batch[0],batch[1],batch[2])

In [ ]:
import hiddenlayer as hl

transforms = [hl.transforms.Prune('Constant')] # Removes Constant nodes from graph.

graph = hl.build_graph(net, (batch[0], batch[1], batch[2]), transforms=transforms)
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('rnn_hiddenlayer_1', format='png')

In [ ]:
criterion = nn.TripletMarginLoss(margin=5.0, p=2)

optimizer = torch.optim.SGD(net.parameters(),
                            lr=0.0005,
                            momentum=0.9,
                            weight_decay=2e-3,#The value used in the paper is 1e-3
                            nesterov=True)

In [ ]:
from torch.autograd import Variable
for epoch in range(1):

        running_loss = 0.0
        loss_train = 0.0
        for batch_idx, (data1, data2, data3) in enumerate(train_loader):

#             if is_gpu:
#                 data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

            # wrap in torch.autograd.Variable
            data1, data2, data3 = Variable(
                data1), Variable(data2), Variable(data3)
            print('anchor', data1.size())
            print('positive', data2.size())
            print('negative', data3.size())

            # compute output and loss
            embedded_a, embedded_p, embedded_n = net(data1, data2, data3)
            loss = criterion(embedded_a, embedded_p, embedded_n)
            print(loss)

            # compute gradient and do optimizer step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # print the loss
            running_loss += loss.data

#             loss_train_cls = torch.sum(
#                 1 * (criterion_val(embedded_a, embedded_p,
#                                    embedded_n) > 0)) / train_batch_size  # CHANGED, MAY NEED TO REVERT BACK

#             loss_train += loss_train_cls.data

            if batch_idx % 30 == 0:
                print("mini Batch Loss: {}".format(loss.data))

In [ ]:
'''
Network constructed
'''
print('================== NETWORK CONSTRUCTED ==================')

In [ ]:
# Loss function and Optimizer
from torch.autograd import Variable
import torch.optim as optim

criterion = nn.TripletMarginLoss(margin=5, p=2, reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [ ]:
def train(model, criterion, optimizer, epochs, trainloader):
    for epoch in range(epochs):
        running_loss = 0
        for batch_idx, (data0, data1, data2) in enumerate(trainloader):
            anchor, positive, negative = data0, data1, data2
#             anchor = Variable(anchor)
#             positive = Variable(positive)
#             negative = Variable(negative)
            print('anchor', anchor.size())
            print('positive', positive.size())
            print('negative', negative.size())
            
            # Calculate the output of three networks
            embedded_a, embedded_p, embedded_n = model(anchor, positive, negative)
            
            # Calculate the loss
            loss = criterion(embedded_a, embedded_p, embedded_n)
            print(loss)
            
            # Zero the gradient
            optimizer.zero_grad()
            
            # Back prop and update
            loss.backward()
            optimizer.step()
            
            # print statistics
            running_loss += loss.item()
            
        print(f'[{epoch + 1}] average loss per epoch: {running_loss / len(train_loader):.3f}')
        # save checkpoint of model
        if epoch % 5 == 0 and epoch > 0:
            save_path = f'model_epoch_{epoch}.pt'
            torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optim.state_dict()}, save_path)
        print(f'Saved model checkpoint to {save_path}')
        
    print('Finished Training')

In [ ]:
train(net, criterion, optimizer, 1, train_loader)